---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [300]:
def date_sorter():
    
    from datetime import datetime
    import numpy as np
    ## 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    ex1_a = df.str.extractall(r'(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{2})\b')
    ex1_b = df.str.extractall(r'(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{4})\b')
    ex1 = pd.concat([ex1_a,ex1_b])
    ex1.reset_index(inplace=True)
    ex1_index = ex1['level_0']

    ## Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    #ex2 = df.str.extractall(r'((?P<Month>:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ,]*)(?P<Day>\d{1,2}[?:, -]*)(?P<Year>\d{4})')
    #ex2.columns = ['Month', 'Day', 'Year']
    ex2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    ex2.reset_index(inplace=True)
    ex2_index = ex2['level_0']

    ## 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    ## Feb 2009; Sep 2009; Oct 2010
    ex3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    ex3.columns = ['Day', 'Month', 'Year']
    ex3.reset_index(inplace=True)
    ex3_index = ex3['level_0']

    ## Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #ex = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z] )((?:\d{1,2}(?:st|nd|rd|th), )\d{4})')

    ## 6/2008; 12/2009
    ex4 = df.str.extractall(r'(?P<Month>\d{1,2})[/](?P<Year>\d{4})')
    ex4.reset_index(inplace=True)
    ex4 = ex4.set_index('level_0')
    ex4 = ex4.loc[~ex4.index.isin(ex1_index)]
    ex4_index = ex4.index

    ## 2009; 2010
    ex5_a = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    ex5_b = df.str.extractall(r'^(\d{4})[^0-9]')
    ex5 = pd.concat([ex5_a,ex5_b])
    ex5.reset_index(inplace=True)
    ex5 = ex5.set_index('level_0')
    ex5 = ex5.loc[~ex5.index.isin(ex1_index)]
    ex5 = ex5.loc[~ex5.index.isin(ex2_index)]
    ex5 = ex5.loc[~ex5.index.isin(ex3_index)]
    ex5 = ex5.loc[~ex5.index.isin(ex4_index)]
    ex5_index = ex5.index


    #combine index into single lsit
    index_list = ex1_index.values.tolist()+ex2_index.values.tolist()+ex3_index.values.tolist()+ex4_index.values.tolist()+ex5_index.values.tolist()
    index_list = np.asarray(index_list)

    # Cleanup ex1
    ex1['Year']=ex1['Year'].apply(str)
    ex1['Year']=ex1['Year'].apply(lambda x: '19'+x if len(x)<=2 else x)
    ex1['Month'] = pd.to_numeric(ex1['Month'])

    # Cleanup ex2
    ex2[1] = ex2[1].apply(lambda x: x.replace(',',''))
    ex2[["Day", "Year"]] = ex2[1].str.split(' ', expand=True, n=1)
    ex2.rename(columns = {0:'Month'}, inplace=True)
    ex2['Month'] = ex2.Month.apply(lambda x: x[:3])
    ex2['Month'] = pd.to_datetime(ex2.Month, format='%b').dt.month
    ex2.drop(1,axis=1,inplace=True) 

    # Cleanup ex3
    ex3['Day'] = ex3['Day'].replace(np.nan,1)
    ex3['Month'] = ex3.Month.apply(lambda x: x[:3])
    ex3['Month'] = pd.to_datetime(ex3.Month, format='%b').dt.month

    # Cleanup ex4
    ex4.reset_index(inplace=True)
    ex4['Day']=1

    # Cleanup ex5
    ex5.reset_index(inplace=True)
    ex5.rename(columns = {0:'Year'}, inplace=True)
    ex5['Day']=1
    ex5['Month']=1

    #Combine
    final_df = pd.concat([ex1,ex2,ex3,ex4,ex5])
    final_df['Day'] = pd.to_numeric(final_df.Day)
    final_df['Date'] = final_df['Day'].apply(str)+'-'+final_df['Month'].apply(str)+'-'+final_df['Year'].apply(str)
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='%d-%m-%Y')
    final_df = final_df.sort_values(by='level_0').set_index('level_0')


    answer = pd.Series([i[0] for i in sorted(enumerate(final_df['Date']), key=lambda x:x[1])],np.arange(500))

    
    return answer